In [1]:
import sys
import django
import os
file_dir = "/Users/mirbilal/Desktop/MobCommission/commissionV2/"
if file_dir not in sys.path:
    sys.path.insert(0, file_dir)

os.environ["DJANGO_SETTINGS_MODULE"] = "commissionerv2.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true" 
django.setup()
from os import path, environ
from datetime import datetime
from services.zayev.environment.market_simulator import MarketSimulator
from services.zayev.service_layer.forester import Forester
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.metrics import accuracy_score
from matplotlib.dates import relativedelta
from apps.environment.service_layer.data_transformer import DataTransformer
import pandas as pd


2023-12-13 11:52:33.275419: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
starting_time_step = datetime(year=2023, month=12, day=11)

db_params = {
    'database': environ.get("POSTGRES_DB"),
    'user': environ.get("POSTGRES_USER"),
    'password': environ.get("POSTGRES_PASSWORD"),
    'host': environ.get("DB_HOST"),
    'port': environ.get("DB_PORT"),
}
env_config = {
    "db_params": db_params, 
    "max_episode_steps": 900, 
    "the_current_time_step": starting_time_step,
    "print_output": False,
    "is_test": False,
    "test_steps": 200,
    "n_step_stocks": 5,
    "n_step_cmmdties": 5,
    "preparing": True,
    "no_of_stocks": 100,
    "no_of_cmmdts": 14,
    "is_live": True
} 
dformer = DataTransformer(env_config=env_config)
dfs = dformer.create_input_dataframes()
display(dfs[2])
# market = MarketSimulator(env_config=env_config, data_frames=dfs)
# market.reset()
# print(market.the_current_time_step)

/Users/mirbilal/Desktop/MobCommission/commissionV2/apps/environment/service_layer/data_transformer.py:269: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  stcks_concatenated_df[field] = stcks_concatenated_df.groupby('index')[field].fillna(method='ffill')
/Users/mirbilal/Desktop/MobCommission/commissionV2/apps/environment/service_layer/data_transformer.py:270: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  stcks_concatenated_df[field] = stcks_concatenated_df.groupby('index')[field].fillna(method='bfill')
/Users/mirbilal/Desktop/MobCommission/commissionV2/apps/environment/service_layer/data_transformer.py:299: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cmmdts_concatenated_df[field] = cmmdts_concatenated_df.groupby('index')[fi

,id,captured_at,price_snapshot,change,volume,bid_vol,bid_price,offer_vol,offer_price,stock_id,close,high,low,open,index
0,50331.0,2023-12-11 00:00:00+00:00,4.09,0.062338,9615500.0,100000000.0,4.09,100000000.0,4.09,1.0,4.09,4.17,3.87,3.87,1
1,50332.0,2023-12-12 00:00:00+00:00,4.02,-0.017115,3492000.0,100000000.0,4.02,100000000.0,4.02,1.0,4.02,4.24,4.00,4.15,1
200,50332.0,2023-12-13 00:00:00+00:00,4.02,0.000000,3492000.0,100000000.0,4.02,100000000.0,4.02,1.0,4.02,4.24,4.00,4.15,1
300,50332.0,2023-12-14 00:00:00+00:00,4.02,0.000000,3492000.0,100000000.0,4.02,100000000.0,4.02,1.0,4.02,4.24,4.00,4.15,1
400,50332.0,2023-12-15 00:00:00+00:00,4.02,0.000000,3492000.0,100000000.0,4.02,100000000.0,4.02,1.0,4.02,4.24,4.00,4.15,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64099,140134.0,2026-05-25 00:00:00+00:00,493.62,0.000000,4000.0,100000000.0,493.62,100000000.0,493.62,100.0,493.62,500.00,492.00,497.00,100
64199,140134.0,2026-05-26 00:00:00+00:00,493.62,0.000000,4000.0,100000000.0,493.62,100000000.0,493.62,100.0,493.62,500.00,492.00,497.00,100
64299,140134.0,2026-05-27 00:00:00+00:00,493.62,0.000000,4000.0,100000000.0,493.62,100000000.0,493.62,100.0,493.62,500.00,492.00,497.00,100
64399,140134.0,2026-05-28 00:00:00+00:00,493.62,0.000000,4000.0,100000000.0,493.62,100000000.0,493.62,100.0,493.62,500.00,492.00,497.00,100


In [3]:
nw_starting_time_step = datetime(year=2023, month=9, day=1)

nw_db_params = {
    'database': environ.get("POSTGRES_DB"),
    'user': environ.get("POSTGRES_USER"),
    'password': environ.get("POSTGRES_PASSWORD"),
    'host': environ.get("DB_HOST"),
    'port': environ.get("DB_PORT"),
}
nw_env_config = {
    "db_params": nw_db_params, 
    "max_episode_steps": 60, 
    "the_current_time_step": nw_starting_time_step,
    "print_output": False,
    "is_test": False,
    "test_steps": 60,
    "n_step_stocks": 5,
    "n_step_cmmdties": 5,
    "preparing": True,
    "no_of_stocks": 100,
    "no_of_cmmdts": 14,
    "is_live": True
} 

nw_market = MarketSimulator(env_config=nw_env_config)
nw_market.reset()
nw_forest = Forester(market=nw_market)
print(nw_market.the_current_time_step)

14
[]
[]
[]
[]
[]
[]
14
[]
[]
[]
[]
[]
[]
2023-09-01 00:00:00+00:00


In [ ]:
forest = Forester(market=market)
forest.make_forest_data()
X_train = forest.train_input
y_train = forest.train_output
y_test = forest.test_output
X_test = forest.test_input
print(forest.market.the_current_time_step)

nw_forest.make_forest_data()
nw_X_train = nw_forest.train_input
nw_y_train = nw_forest.train_output
nw_y_test = nw_forest.test_output
nw_X_test = nw_forest.test_input
print(nw_forest.market.the_current_time_step)